In [5]:
import pandas as pd
from googleapiclient.discovery import build

# replace with your own API key
API_KEY = 'Your_API_KEY'

def get_trending_videos(api_key, max_results=200):
    # build the youtube service
    youtube = build('youtube', 'v3', developerKey=api_key)

    # initialize the list to hold video details
    videos = []

    # fetch the most popular videos
    request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        chart='mostPopular',
        regionCode='IN',
        maxResults=50
    )

    # paginate through the results if max_results > 50
    while request and len(videos) < max_results:
        try:
            response = request.execute()
            print(f'Fetched {len(response["items"])} videos in this batch.')  # Debug line
            
            for item in response['items']:
                video_details = {
                    'video_id': item['id'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_id': item['snippet']['channelId'],
                    'channel_title': item['snippet']['channelTitle'],
                    'category_id': item['snippet']['categoryId'],
                    'tags': item['snippet'].get('tags', []),
                    'duration': item['contentDetails']['duration'],
                    'definition': item['contentDetails']['definition'],
                    'caption': item['contentDetails'].get('caption', 'false'),
                    'view_count': item['statistics'].get('viewCount', 0),
                    'like_count': item['statistics'].get('likeCount', 0),
                    'dislike_count': item['statistics'].get('dislikeCount', 0),
                    'favorite_count': item['statistics'].get('favoriteCount', 0),
                    'comment_count': item['statistics'].get('commentCount', 0)
                }
                videos.append(video_details)

            # Debug: Check if there is a next page token
            if 'nextPageToken' in response:
                print('There are more videos to fetch.')
                request = youtube.videos().list_next(request, response)
            else:
                print('No more videos to fetch.')
                break  # Exit if there's no next page token

        except Exception as e:
            print(f'An error occurred: {e}')
            break  # Exit the loop on error

    return videos[:max_results]

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

def main():
    trending_videos = get_trending_videos(API_KEY)
    filename = 'trending_videos.csv'
    save_to_csv(trending_videos, filename)
    print(f'Trending videos saved to {filename}')

if __name__ == '__main__':
    main()


Fetched 50 videos in this batch.
There are more videos to fetch.
Fetched 50 videos in this batch.
There are more videos to fetch.
Fetched 50 videos in this batch.
There are more videos to fetch.
Fetched 8 videos in this batch.
No more videos to fetch.
Trending videos saved to trending_videos.csv
